In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.embeddings import FlairEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings,TransformerWordEmbeddings
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.models import TARSClassifier
from flair.trainers import ModelTrainer

## LOAD DATASET

In [2]:
columns = {0: 'text', 1: 'ner'}


data_folder = r'C:/Users/Dawid/Downloads/lecture_data/data/corpus4'

label_name_map = {'FUND': 'Funding Agency',
                  'IND': 'Person',
                  'COR': 'Corporation',
                  'GRNB': 'Grant Number',
                  'UNI': 'University',
                  'MISC': 'Miscellaneous'
                  }


corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
label_type = 'ner'
label_dict = corpus.make_label_dictionary(label_type=label_type)

2023-11-21 21:17:38,122 Reading data from C:\Users\Dawid\Downloads\lecture_data\data\corpus4
2023-11-21 21:17:38,123 Train: C:\Users\Dawid\Downloads\lecture_data\data\corpus4\train.txt
2023-11-21 21:17:38,124 Dev: C:\Users\Dawid\Downloads\lecture_data\data\corpus4\dev.txt
2023-11-21 21:17:38,124 Test: C:\Users\Dawid\Downloads\lecture_data\data\corpus4\test.txt
2023-11-21 21:17:38,413 Computing label dictionary. Progress:


0it [00:00, ?it/s]
1145it [00:00, 88076.85it/s]

2023-11-21 21:17:38,430 Dictionary created for label 'ner' with 6 values: FUND (seen 1313 times), GRNB (seen 1193 times), IND (seen 854 times), UNI (seen 364 times), MISC (seen 207 times), COR (seen 49 times)


# Stacked embedding

In [3]:
stacked_embeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'),
                                        FlairEmbeddings('news-forward'),
                                        FlairEmbeddings('news-backward'),
                                       ])

In [4]:
model = SequenceTagger(hidden_size=256,
                        embeddings=stacked_embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type)

trainer = ModelTrainer(model, corpus)

trainer.train('resources/taggers/ner',
              learning_rate=0.1,
              mini_batch_size=32,
              mini_batch_chunk_size=8,
              max_epochs=10)

2023-11-21 21:17:42,715 SequenceTagger predicts: Dictionary with 25 tags: O, S-FUND, B-FUND, E-FUND, I-FUND, S-GRNB, B-GRNB, E-GRNB, I-GRNB, S-IND, B-IND, E-IND, I-IND, S-UNI, B-UNI, E-UNI, I-UNI, S-MISC, B-MISC, E-MISC, I-MISC, S-COR, B-COR, E-COR, I-COR
2023-11-21 21:17:42,783 ----------------------------------------------------------------------------------------------------
2023-11-21 21:17:42,783 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.89s/it]

2023-11-21 21:20:57,712 DEV : loss 1.3003181219100952 - f1-score (micro avg)  0.1885
2023-11-21 21:20:57,722  - 0 epochs without improvement
2023-11-21 21:20:57,723 saving best model


2023-11-21 21:20:58,246 ----------------------------------------------------------------------------------------------------
2023-11-21 21:21:00,012 epoch 2 - iter 3/36 - loss 1.14554643 - time (sec): 1.77 - samples/sec: 1592.84 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:01,847 epoch 2 - iter 6/36 - loss 1.14709899 - time (sec): 3.60 - samples/sec: 1602.48 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:03,621 epoch 2 - iter 9/36 - loss 1.14915470 - time (sec): 5.37 - samples/sec: 1607.20 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:06,336 epoch 2 - iter 12/36 - loss 1.15108691 - time (sec): 8.09 - samples/sec: 1507.43 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:08,196 epoch 2 - iter 15/36 - loss 1.13448815 - time (sec): 9.95 - samples/sec: 1525.29 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:09,832 epoch 2 - iter 18/36 - loss 1.10985330 - time (sec): 11.59 - samples/sec: 1549.97 - lr: 0.100000 - momentum: 0.000000
2023-11-21 21:21:11,477 epoch 2

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.75it/s]

...

2023-11-21 21:24:43,796 DEV : loss 0.2644353210926056 - f1-score (micro avg)  0.7294
2023-11-21 21:24:43,806  - 2 epochs without improvement


2023-11-21 21:24:44,347 ----------------------------------------------------------------------------------------------------
2023-11-21 21:24:44,347 Loading model from best epoch ...
2023-11-21 21:24:44,931 SequenceTagger predicts: Dictionary with 27 tags: O, S-FUND, B-FUND, E-FUND, I-FUND, S-GRNB, B-GRNB, E-GRNB, I-GRNB, S-IND, B-IND, E-IND, I-IND, S-UNI, B-UNI, E-UNI, I-UNI, S-MISC, B-MISC, E-MISC, I-MISC, S-COR, B-COR, E-COR, I-COR, <START>, <STOP>


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:20<00:00,  6.81s/it]

2023-11-21 21:25:05,481 
Results:
- F-score (micro) 0.7312
- F-score (macro) 0.5079
- Accuracy 0.6031

By class:
              precision    recall  f1-score   support

         IND     0.9183    0.9525    0.9351       295
        GRNB     0.7500    0.8812    0.8103       160
        FUND     0.5635    0.6497    0.6036       157
         UNI     0.5049    0.5253    0.5149        99
        MISC     0.5625    0.1098    0.1837        82
         COR     0.0000    0.0000    0.0000        12

   micro avg     0.7358    0.7267    0.7312       805
   macro avg     0.5499    0.5197    0.5079       805
weighted avg     0.7149    0.7267    0.7035       805

2023-11-21 21:25:05,481 ----------------------------------------------------------------------------------------------------


{'test_score': 0.73125}

# XLNet

In [3]:
embedding = TransformerWordEmbeddings('xlnet-base-cased')
stacked_embeddings = StackedEmbeddings([
                                        embedding,
                                       ])

In [4]:
model = SequenceTagger(hidden_size=256,
                        embeddings=stacked_embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type)

# 6. initialize trainer
trainer = ModelTrainer(model, corpus)

# 7. start training
trainer.train('resources/taggers/ner',
              learning_rate=0.1,
              mini_batch_size=32,
              mini_batch_chunk_size=8,
              max_epochs=10)

2023-11-21 20:08:06,209 SequenceTagger predicts: Dictionary with 25 tags: O, S-IND, B-IND, E-IND, I-IND, S-FUND, B-FUND, E-FUND, I-FUND, S-GRNB, B-GRNB, E-GRNB, I-GRNB, S-UNI, B-UNI, E-UNI, I-UNI, S-MISC, B-MISC, E-MISC, I-MISC, S-COR, B-COR, E-COR, I-COR
2023-11-21 20:08:06,221 ----------------------------------------------------------------------------------------------------
2023-11-21 20:08:06,222 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): XLNetModel(
        (word_embedding): Embedding(32001, 768)
        (layer): ModuleList(
          (0-11): 12 x XLNetLayer(
            (rel_attn): XLNetRelativeAttention(
              (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (ff): XLNetFeedForward(
              (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (layer_1): Line

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.51s/it]

2023-11-21 20:09:46,145 DEV : loss 1.8558348417282104 - f1-score (micro avg)  0.0
2023-11-21 20:09:46,151  - 0 epochs without improvement
2023-11-21 20:09:46,151 ----------------------------------------------------------------------------------------------------


2023-11-21 20:09:52,164 epoch 2 - iter 1/11 - loss 1.85528871 - time (sec): 6.01 - samples/sec: 202.26 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:09:57,086 epoch 2 - iter 2/11 - loss 2.10442218 - time (sec): 10.93 - samples/sec: 214.73 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:00,559 epoch 2 - iter 3/11 - loss 2.07184394 - time (sec): 14.41 - samples/sec: 230.99 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:07,080 epoch 2 - iter 4/11 - loss 2.07246581 - time (sec): 20.93 - samples/sec: 214.35 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:10,408 epoch 2 - iter 5/11 - loss 2.03710311 - time (sec): 24.26 - samples/sec: 229.30 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:46,561 epoch 2 - iter 6/11 - loss 2.30412519 - time (sec): 60.41 - samples/sec: 120.21 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:51,802 epoch 2 - iter 7/11 - loss 2.30844110 - time (sec): 65.65 - samples/sec: 129.05 - lr: 0.100000 - momentum: 0.000000
2023-11-21 20:10:55,2

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.28it/s]

...

2023-11-21 20:21:44,188 DEV : loss 1.9304167032241821 - f1-score (micro avg)  0.0
2023-11-21 20:21:44,193  - 1 epochs without improvement


2023-11-21 20:21:44,506 ----------------------------------------------------------------------------------------------------
2023-11-21 20:21:44,507 Testing using last state of model ...


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:37<00:00, 52.62s/it]

2023-11-21 20:24:22,371 
Results:
- F-score (micro) 0.0821
- F-score (macro) 0.028
- Accuracy 0.0475

By class:
              precision    recall  f1-score   support

        FUND     0.1126    0.3312    0.1680       157
         IND     0.0000    0.0000    0.0000       295
        GRNB     0.0000    0.0000    0.0000       160
         UNI     0.0000    0.0000    0.0000        99
        MISC     0.0000    0.0000    0.0000        82
         COR     0.0000    0.0000    0.0000        12

   micro avg     0.1126    0.0646    0.0821       805
   macro avg     0.0188    0.0552    0.0280       805
weighted avg     0.0220    0.0646    0.0328       805

2023-11-21 20:24:22,371 ----------------------------------------------------------------------------------------------------


{'test_score': 0.08208366219415943}

# TARS

In [3]:
tars = TARSClassifier.load("tars-base")
tars.add_and_switch_to_new_task(task_name="question classification",
                                label_dictionary=label_dict,
                                label_type=label_type,
                                )

2023-11-21 21:06:03,740 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model


In [4]:
trainer = ModelTrainer(tars, corpus)

In [ ]:
trainer.train(base_path='resources/taggers/trec',  # path to store the model artifacts
              learning_rate=0.01,  # use very small learning rate
              mini_batch_size=64,
              mini_batch_chunk_size=4,  # optionally set this if transformer is too much for your machine
              max_epochs=10,  # terminate after 10 epochs
              )

2023-11-21 21:06:03,801 ----------------------------------------------------------------------------------------------------
2023-11-21 21:06:03,802 Model: "TARSClassifier(
  (tars_model): TextClassifier(
    (embeddings): TransformerDocumentEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Lin

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:21<00:00,  7.13s/it]

2023-11-21 21:12:25,378 DEV : loss 0.5616104602813721 - f1-score (micro avg)  0.0
2023-11-21 21:12:25,381  - 0 epochs without improvement
2023-11-21 21:12:25,382 ----------------------------------------------------------------------------------------------------


2023-11-21 21:12:44,534 epoch 2 - iter 1/18 - loss 0.40945269 - time (sec): 19.13 - samples/sec: 16.57 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:12:57,779 epoch 2 - iter 2/18 - loss 0.39493421 - time (sec): 32.37 - samples/sec: 19.61 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:13:15,389 epoch 2 - iter 3/18 - loss 0.37337932 - time (sec): 49.98 - samples/sec: 19.37 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:13:37,062 epoch 2 - iter 4/18 - loss 0.36667196 - time (sec): 71.66 - samples/sec: 17.81 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:13:52,457 epoch 2 - iter 5/18 - loss 0.37031847 - time (sec): 87.05 - samples/sec: 18.10 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:14:08,492 epoch 2 - iter 6/18 - loss 0.36661892 - time (sec): 103.09 - samples/sec: 18.42 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:14:32,579 epoch 2 - iter 7/18 - loss 0.35784856 - time (sec): 127.17 - samples/sec: 17.50 - lr: 0.010000 - momentum: 0.000000
2023-11-21 21:14:49,906 e